In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats
from ta.momentum import RSIIndicator


In [30]:
from ta.volatility import BollingerBands
def BB(window_para, window_dev_para):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        result.loc[code== result['code'].shift(window_para),'bbm'] = BollingerBands(close=close_weekly,window=window_para,window_dev=window_dev_para).bollinger_mavg()
        result.loc[code == result['code'].shift(window_para),'bbh'] = BollingerBands(close=close_weekly,window=window_para,window_dev=window_dev_para).bollinger_hband()
        result.loc[code == result['code'].shift(window_para),'bbl'] = BollingerBands(close=close_weekly,window=window_para,window_dev=window_dev_para).bollinger_lband()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']>result['bbh']),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']<result['bbm']),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    result.loc[(result['signal'].shift(1)==1)&(result['signal']!=1),'counter']=1
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    result.to_csv('BB_Result.csv')
    
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, window_dev_para,max(dd_arr),  result['counter'].sum()
#     return day_return

    

In [36]:
BB(14, 3.2)

MDD 0.11648606209088803


(8.25116740020023, 14, 3.2, 0.11648606209088803, 268.0)

In [34]:
optimize_result = []
for window_para in range(2,20, 1):
    for window_dev_para in range(20,40,2):
        x=BB(window_para, window_dev_para/10)
        optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.5084399843911295
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.2246788119068106
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.05071906890426736
MDD 0.3460140605729197
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.040718739113696646
MDD 0.25798083178409376
MDD 0.3928629849826258
MDD 0

In [35]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2,3,4
116,10.855746,13,3.2,0.114413,143.0
167,8.631554,18,3.4,0.160944,311.0
105,8.608341,12,3.0,0.135816,313.0
126,8.251167,14,3.2,0.116486,268.0
146,7.899973,16,3.2,0.187951,416.0
94,7.330743,11,2.8,0.235764,582.0
156,7.157452,17,3.2,0.243169,490.0
136,7.076351,15,3.2,0.170050,351.0
104,7.004193,12,2.8,0.234704,735.0
177,6.931072,19,3.4,0.197009,357.0


In [29]:
checkbb[checkbb['code']=='AMD']

NameError: name 'checkbb' is not defined

In [54]:
from ta.momentum import RSIIndicator
def RSI(window_para, highband, to_csv):
    lowband = 100 - highband
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    result.loc[result['code'] == result['code'].shift(window_para),'rsi'] = RSIIndicator(close=result['close_weekly'],window=window_para).rsi()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result.loc[result['code'] == result['code'].shift(window_para),'change_code'] = 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['rsi']<lowband),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['rsi']>highband),'signal'] = 0
    result = result.ffill()
    result['daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkrsi.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, highband

In [55]:
RSI(18, 86, to_csv = False)

MDD 0.2682730484377789


(7.282815260811104, 18, 86)

In [62]:
from ta.momentum import StochasticOscillator
def Stochastic(window_para,smwd_para, highband, to_csv):
    lowband = 100 - highband
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    result.loc[result['code'] == result['code'].shift(window_para),'stoch'] = StochasticOscillator(high = result['high_weekly'], low = result['low_weekly'], close=result['close_weekly'],window=window_para, smooth_window = smwd_para).stoch()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result.loc[result['code'] == result['code'].shift(window_para),'change_code'] = 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['stoch']<lowband),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['stoch']>highband),'signal'] = 0
    result = result.ffill()
    result['daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkstock.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, highband, smwd_para

In [63]:
from ta.trend import SMAIndicator
def SMA(window_para,to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    result.loc[result['code'] == result['code'].shift(window_para),'sma'] = SMAIndicator(close=result['close_weekly'],window=window_para).sma_indicator()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result.loc[result['code'] == result['code'].shift(window_para),'change_code'] = 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']>result['sma']),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']<result['sma']),'signal'] = 0
    result = result.ffill()
    result['daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checksma.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para

In [64]:
from ta.trend import EMAIndicator
def EMA(window_para,to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    result.loc[result['code'] == result['code'].shift(window_para),'ema'] = EMAIndicator(close=result['close_weekly'],window=window_para).ema_indicator()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result.loc[result['code'] == result['code'].shift(window_para),'change_code'] = 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']>result['ema']),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']<result['ema']),'signal'] = 0
    result = result.ffill()
    result['daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checksma.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para

In [65]:
Stochastic(10,3, 54, to_csv =False)

MDD 0.19401886637417048


(4.517097235936747, 10, 54, 3)

In [66]:
SMA(42,to_csv = True)

MDD 0.3541840366238755


(1.9728942241154983, 42)

In [67]:
EMA(33,to_csv = True)

MDD 0.3704671977107173


(1.9760422832528357, 33)

In [41]:
optimize_result = []
for window_para in range(2,50, 1):
    x=SMA(window_para,to_csv = False)
    optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

In [24]:
optimize_result = []
for window_para in range(2,30, 2):
    for smwd_para in range(10,50,10):
        for highband in range(50,100, 5):
            x=Stochastic(window_para,smwd_para/10, highband, to_csv = False)
            optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

In [42]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1
40,1.972894,42
43,1.961611,45
39,1.958651,41
45,1.949790,47
44,1.948883,46
35,1.944000,37
41,1.943328,43
47,1.942670,49
46,1.940307,48
34,1.935724,36


In [22]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2
118,4.661947,2,86
93,4.661947,2,86
193,4.661947,2,86
43,4.661947,2,86
168,4.661947,2,86
143,4.661947,2,86
18,4.661947,2,86
68,4.661947,2,86
144,4.651083,2,88
19,4.651083,2,88


In [48]:
day_return.to_csv('checkBB.csv')